# Set up the environnement

Used to test BERT on GLUE subsets using huggingface bibli

In [ ]:
!pip install -r requirements.txt
!git clone 
!cd MLAProject
!cd main

In [2]:
from Class_Fine_Tuning_V1 import *
import os
import numpy as np
import json
import sys
import matplotlib.pyplot as plt

sys.path.append('/ProjetMLA/main/modified_transformers')
from modified_transformers import BertForSequenceClassification,BertConfig,AutoTokenizer
from datasets import load_dataset
from torch.optim import AdamW

# Data preparation

In [12]:
glue_data = load_dataset('glue','cola')

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


print(glue_data['train'].features)
print(glue_data["train"][0])

token = tokenizer(glue_data['train']['sentence'])
token[0].ids


{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(names=['unacceptable', 'acceptable'], id=None), 'idx': Value(dtype='int32', id=None)}
{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.", 'label': 1, 'idx': 0}


[101,
 3458,
 2053,
 1281,
 112,
 189,
 4417,
 1142,
 3622,
 117,
 1519,
 2041,
 1103,
 1397,
 1141,
 1195,
 17794,
 119,
 102]

The Autokenizer is a class from transformers bibli which transform a sentences into a vector understood by the bert model.

In [ ]:
config = BertConfig(hidden_size = 1024,
                    max_position_embeddings=512,
                    num_attention_heads=16,
                    num_hidden_layers = 24)
model1 = BertForSequenceClassification(config)

FineTuning = Fine_Tuning(model1, tokenizer)
FineTuning.load_data(dataset_size=5000,batch_size=32)

We have created a method named load_data that allows loading the data, splitting the dataset into training, validation, and test sets, and further dividing these datasets into batches.
The method take in argument the number of example in the training dataset and the size of a batch

# Configuration of a model

In [15]:
config = BertConfig(hidden_size = 1024,
                    max_position_embeddings=512,
                    num_attention_heads=16,
                    num_hidden_layers = 24)
model1 = BertForSequenceClassification(config)
print(config)
config = BertConfig(adapter = True,
                    frozen_mode =True,
                    hidden_size = 1024,
                    max_position_embeddings=512,
                    num_attention_heads=16,
                    num_hidden_layers = 24)

model2 = BertForSequenceClassification(config)
print(config)

BertConfig {
  "adapter": false,
  "adapter_size": 16,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "frozen_mode": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

BertConfig {
  "adapter": true,
  "adapter_size": 16,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "frozen_mode": true,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden

Thanks to the modified transfomers module that we created from the transformers bibli, we can create different bert model with or not adapter layer. The function Config can take in arguments the size of an adapter, if the parameter of bert model have to be frozen.

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-5)
FineTuning.initialisation(optimizer,epoch=1)
FineTuning.training()

Here, thanks to the class FineTuning, we can train the bert model on the dataset cola for example.
We can specify the number of epoch

# Experiment Execution:

In [ ]:
!python /ProjetMLA/main/Fine_Tuning_lr.py

Fine_Tuning_lr script is designed for fine-tuning the BERT model with different learning rates, aiding in the search for optimal hyperparameters.

In [ ]:
!python /ProjetMLA/main/Fine_Tuning.py

Thanks to Fine_Tuning script we can fine-tune the BERT model on different GLUE datasets.

But watch out the path where you want to save your results

# Loading and Plot the results


In [ ]:
file = '/ProjetMLA/admin/Resultats/Run_cola/'
data = []
file_name = []
for files in os.listdir(file):
    if files.endswith('.npy'):
        file_name.append(files)
        path = os.path.join(file,files)
        data.append(np.load(path,allow_pickle=True))

Here we load the results

In [16]:
for i in range(len(data)):
    plt.plot(data[i][0],label="train_accuracy")
    plt.plot(data[i][1],label="val_accuracy")
    plt.title("Accuracy on "+file_name[i])
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("Accuracy")
    plt.show()
for i in range(len(data)):
    plt.plot(data[i][3],label="train_loss")
    plt.plot(data[i][4],label="val_loss")
    plt.title("loss on "+file_name[i])
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.show()
for i in range(len(data)):
    print('Accuracy at the end of fine tuning on {}: {}'.format(file_name[i],data[i][2][1]))

NameError: name 'data' is not defined